# LeNet-5

**Korzystając z notebooka o sieciach konwolucyjnych oraz notebooka o płytkiej sieci w Tensorflow, za pomocą biblioteki Tensorflow zaprojektuj konwolucyjną sieć neuronową o architekturze zbliżonej do słynnej sieci [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) której sukcesy w rozpoznawaniu obrazów rozpoczęły na przełomie lat 80. i 90. boom na głębokie uczenie, i użyj jej do predykcji zbioru FashionMnist, zgodnie z następującym opisem:**

|Nr warstwy|Rodzaj|Rozmiar wyjścia|Liczba filtrów|Rozmiar jądra (`kernel_size`)|Krok (`stride`)|Padding|Funkcja aktywacji|
|----------|---|---------------|--------------|-----------------------|---------------|-------|-----------------|
|1|splotowa|$28\times 28$|6|$5\times 5$|1|2|tanh|
|2|avgerage pooling|$14\times 14$|6|$2\times 2$|2|0|sigmoid|
|3|splotowa|$10\times 10$|16|$5\times 5$|1|0|tanh|
|4|average pooling|$5\times 5$|16|$2\times 2$|2|0|sigmoid|
|5|splotowa|$1\times 1$|120|$5\times 5$|1|0|tanh|
|6|pełna|84|||||tanh|
|7|pełna|10|||||softmax|

**Kolejne kroki:**
 - załadowanie zbioru
 - wydzielenie zbioru walidacyjnego
 - normalizacja i kategoryzacja danych
 - zaprojektowanie sieci (nie zapominaj o spłaszczeniu w odpowiednim momencie)
 - zbudowanie modelu z dowolną funkcją kosztu dla problemów wieloklasowych, dowolnym optymalizatorem i trafnością jako metryką
 - nauczenie sieci w maksymalnie stu epokach z zastosowaniem wczesnego zatrzymania (najlepiej użyć wbudowanego, z dowolną wartością do monitorowania)
 - ewaluacja modelu na zbiorze walidacyjnym
 - wyświetlenie 10 niepoprawnie zaklasyfikowanych próbek w postaci: prawdziwa prognozowana, np. 2 7